In [58]:
import os
import re
import subprocess
from pathlib import Path

import pandas as pd

from wombat.bsub import batch_bsub_commands, write_command_file, DEFAULT_ARGS
from wombat.utils import listfiles

In [59]:
run_dir = '/scratch1/fs1/dinglab/estorrs/ancestry/executions/gbm_test'
Path(run_dir).mkdir(parents=True, exist_ok=True)

## define inputs

thousand genomes sample panel used for training

In [60]:
thousand_genomes_panel = '/storage1/fs1/dinglab/Active/Projects/Ancestry_Data/integrated_call_samples_v3.20130502.ALL.panel'

reference genome (preferably the same reference used to align the .bams that are having ancestry called on them)

In [61]:
reference_fasta = '/storage1/fs1/dinglab/Active/Projects/PECGS/ref_genome/GRCh38.d1.vd1.fa'

vcf and bed file for thousand genomes project samples.

make sure to use the correct vcf/bed file depending on your reference (hg19 vs hg38) and whether the genome coordinates start with "chr" or not

you can find different versions of these files here

hg38 and genome coordinates start with chr
+ thousand_genomes_vcf = '/storage1/fs1/dinglab/Active/Projects/Ancestry_Data/hg38/all.coding.sorted.02maf.snps.chr.vcf'
+ thousand_genomes_bed = '/storage1/fs1/dinglab/Active/Projects/Ancestry_Data/hg38/all.coding.sorted.02maf.snps.chr.bed'

hg38 and genome coordinates do not start with chr
+ thousand_genomes_vcf = '/storage1/fs1/dinglab/Active/Projects/Ancestry_Data/hg38/all.coding.sorted.02maf.snps.vcf'
+ thousand_genomes_bed = '/storage1/fs1/dinglab/Active/Projects/Ancestry_Data/hg38/all.coding.sorted.02maf.snps.bed'

hg19 and genome coordinates start with chr
+ thousand_genomes_vcf = '/storage1/fs1/dinglab/Active/Projects/Ancestry_Data/hg19/all.coding.sorted.02maf.snps.chr.vcf'
+ thousand_genomes_bed = '/storage1/fs1/dinglab/Active/Projects/Ancestry_Data/hg19/all.coding.sorted.02maf.snps.chr.bed'

hg19 and genome coordinates do not start with chr
+ thousand_genomes_vcf = '/storage1/fs1/dinglab/Active/Projects/Ancestry_Data/hg19/all.coding.sorted.02maf.snps.vcf'
+ thousand_genomes_bed = '/storage1/fs1/dinglab/Active/Projects/Ancestry_Data/hg19/all.coding.sorted.02maf.snps.bed'

In [62]:
thousand_genomes_vcf = '/storage1/fs1/dinglab/Active/Projects/Ancestry_Data/hg38/all.coding.sorted.02maf.snps.chr.vcf'
thousand_genomes_bed = '/storage1/fs1/dinglab/Active/Projects/Ancestry_Data/hg38/all.coding.sorted.02maf.snps.chr.bed'

sample bam locations

In [63]:
df = pd.read_csv('/storage1/fs1/dinglab/Active/Projects/estorrs/ancestry-pipeline/executions/cptac_test/gbm_sample_to_bam.txt',
                sep='\t')
df

,sample_id,filepath
0,C1230738.WXS.T.ADNA_eb44394c.hg38,/storage1/fs1/dinglab/Active/Primary/CPTAC3.sh...
1,C1230738.WXS.T.hg38,/storage1/fs1/dinglab/Active/Primary/CPTAC3.sh...
2,C1245129.WXS.T.ADNA_f4f0a623.hg38,/storage1/fs1/dinglab/Active/Primary/CPTAC3.sh...
3,C1245129.WXS.T.hg38,/storage1/fs1/dinglab/Active/Primary/CPTAC3.sh...
4,C204057.WXS.T.ADNA_fb79d37d.hg38,/storage1/fs1/dinglab/Active/Primary/CPTAC3.sh...
...,...,...
64,C761370.WXS.T.ADNA_260f1df4.hg38,/storage1/fs1/dinglab/Active/Primary/CPTAC3.sh...
65,C761370.WXS.T.hg38,/storage1/fs1/dinglab/Active/Primary/CPTAC3.sh...
66,C827913.WXS.T.hg38,/storage1/fs1/dinglab/Active/Primary/CPTAC3.sh...
67,C846363.WXS.T.ADNA_51dcc4a3.hg38,/storage1/fs1/dinglab/Active/Primary/CPTAC3.sh...


sanity check to make sure genome coordinates match, in this test case we expect "chr" to prepend all genome coordinates

In [64]:
pd.read_csv(thousand_genomes_bed, sep='\t', header=None, nrows=1)

,0,1,2
0,chr1,13273,13273


In [65]:
fp = df['filepath'][0]
cmd = f'samtools view {fp} | head -n 1'
out = subprocess.check_output(cmd, shell=True) # chromosome coordinate is in third column
out.decode().split('\t')[2]

'chr1'

## setting bsub related parameters

## step 1: extract readcounts with bam-readcount

In [66]:
execution_dir = os.path.join(run_dir, '1.slate')
log_dir = os.path.join(execution_dir, 'logs')
result_dir = os.path.join(execution_dir, 'results')
Path(log_dir).mkdir(parents=True, exist_ok=True)
Path(result_dir).mkdir(parents=True, exist_ok=True)

In [67]:
def get_slate_command(input_bam_fp, readcount_fp, trimmed_bam_fp):
    return f'python /slate/slate/slate.py --min-base-quality 20 --min-mapping-quality 20 \
--fasta {reference_fasta} --positions {thousand_genomes_bed} \
--readcount-output {readcount_fp} --filtered-bam-output {trimmed_bam_fp} {input_bam_fp}'

In [68]:
volumes = ['/storage1/fs1/dinglab', '/scratch1/fs1/dinglab', '/home/estorrs'] # compute1 needs to know which directories to map, replace /home/<username> if you are not me
exports = ['/miniconda/bin'] # need to override compute1 PATH defaults
args = DEFAULT_ARGS.copy()
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'gpu_model': 'TeslaV100_SXM2_32GB',
 'gpu_mem': '30',
 'gpu_num': 1,
 'use_gpu': False,
 'group': 'compute-dinglab',
 'group_name': None,
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [69]:
args['username'] = 'estorrs' # change username if you arent me :) 
args['docker'] = 'estorrs/slate:0.0.2'
args['queue'] = 'general'
args['n_concurrent'] = 100
args['mem'] = 5
args['group_name'] = 'ancestry_slate' # job group name

commands = []
for s_id, fp in zip(df['sample_id'], df['filepath']):

    command = get_slate_command(fp,
                                os.path.join(result_dir, f'{s_id}.readcount'),
                                os.path.join(result_dir, f'{s_id}.filtered.bam'))

    commands.append(command)
                     
bsub_commands = batch_bsub_commands(commands, df['sample_id'].to_list(), log_dir, args,
                                    volumes=volumes, exports=exports)
bsub_commands

['mkdir -p /scratch1/fs1/dinglab/estorrs/ancestry/executions/gbm_test/1.slate/logs',
 'export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab:/storage1/fs1/dinglab /scratch1/fs1/dinglab:/scratch1/fs1/dinglab /home/estorrs:/home/estorrs /scratch1/fs1/dinglab/estorrs/ancestry/executions/gbm_test/1.slate/logs:/scratch1/fs1/dinglab/estorrs/ancestry/executions/gbm_test/1.slate/logs"',
 'bgadd -L 100 /estorrs/ancestry_slate',
 'export PATH="/opt/java/openjdk/bin:/miniconda/bin:$PATH"',
 "bsub -R 'select[mem>5GB] rusage[mem=5GB] span[hosts=1]' -M 6GB -n 1 -q general -G compute-dinglab -a 'docker(estorrs/slate:0.0.2)' -g /estorrs/ancestry_slate -J C1230738.WXS.T.ADNA_eb44394c.hg38 -oo /scratch1/fs1/dinglab/estorrs/ancestry/executions/gbm_test/1.slate/logs/C1230738.WXS.T.ADNA_eb44394c.hg38.txt 'python /slate/slate/slate.py --min-base-quality 20 --min-mapping-quality 20 --fasta /storage1/fs1/dinglab/Active/Projects/PECGS/ref_genome/GRCh38.d1.vd1.fa --positions /storage1/fs1/dinglab/Active/Projects/Anc

In [71]:
fp = os.path.join(execution_dir, 'run_slate.sh')
write_command_file(bsub_commands, fp)
fp

'/scratch1/fs1/dinglab/estorrs/ancestry/executions/gbm_test/1.slate/run_slate.sh'

sanity check to make sure all runs completed successfully

In [73]:
fps = sorted(listfiles(log_dir, regex=r'.txt$'))
count = 0
for fp in fps:
    f = open(fp)
    if 'Successfully completed.' in f.read():
        count += 1
    f.close()
len(fps), count

(69, 69)

After running, make sure to clean up slate directory, it produces trim bammed as an output and they take up a lot of space. run the below command to delete the unecessary fiels

In [75]:
fps = sorted(listfiles(result_dir, regex=r'.filtered.bam$'))
len(fps)

69

In [76]:
for fp in fps:
    os.remove(fp)

## step 2: run genotype caller

In [77]:
execution_dir = os.path.join(run_dir, '2.genotype_caller')
log_dir = os.path.join(execution_dir, 'logs')
result_dir = os.path.join(execution_dir, 'results')
Path(log_dir).mkdir(parents=True, exist_ok=True)
Path(result_dir).mkdir(parents=True, exist_ok=True)

In [78]:
def get_genotype_calling_command(script, readcount_dir, output_fp):
    return f'python  {script} \
--readcount-dir {readcount_dir} --genomes-vcf {thousand_genomes_vcf} \
--output {output_fp}'

In [79]:
volumes = ['/storage1/fs1/dinglab', '/scratch1/fs1/dinglab', '/home/estorrs'] # compute1 needs to know which directories to map, replace /home/<username> if you are not me
exports = ['/miniconda/envs/ancestry/bin'] # need to override compute1 PATH defaults
args = DEFAULT_ARGS.copy()
args

{'mem': 10,
 'n_processes': 1,
 'max_mem': None,
 'docker': 'python:3.8',
 'queue': 'dinglab',
 'gpu_model': 'TeslaV100_SXM2_32GB',
 'gpu_mem': '30',
 'gpu_num': 1,
 'use_gpu': False,
 'group': 'compute-dinglab',
 'group_name': None,
 'n_concurrent': 10,
 'interactive': False,
 'username': 'estorrs'}

In [81]:
args['username'] = 'estorrs' # change username if you arent me :) 
args['docker'] = 'estorrs/ancestry-pipeline:0.0.1'
args['queue'] = 'general'
args['mem'] = 100

command = get_genotype_calling_command(
    '/ancestry-pipeline/ancestry/readcount_caller.py',
    os.path.join(run_dir, '1.slate', 'results'),
    os.path.join(result_dir, 'output.vcf'))

                     
bsub_commands = batch_bsub_commands([command], 'call readcounts', log_dir, args, volumes=volumes, exports=exports)
bsub_commands

['mkdir -p /scratch1/fs1/dinglab/estorrs/ancestry/executions/gbm_test/2.genotype_caller/logs',
 'export LSF_DOCKER_VOLUMES="/storage1/fs1/dinglab:/storage1/fs1/dinglab /scratch1/fs1/dinglab:/scratch1/fs1/dinglab /home/estorrs:/home/estorrs /scratch1/fs1/dinglab/estorrs/ancestry/executions/gbm_test/2.genotype_caller/logs:/scratch1/fs1/dinglab/estorrs/ancestry/executions/gbm_test/2.genotype_caller/logs"',
 'export PATH="/opt/java/openjdk/bin:/miniconda/envs/ancestry/bin:$PATH"',
 "bsub -R 'select[mem>100GB] rusage[mem=100GB] span[hosts=1]' -M 101GB -n 1 -q general -G compute-dinglab -a 'docker(estorrs/ancestry-pipeline:0.0.1)' -oo /scratch1/fs1/dinglab/estorrs/ancestry/executions/gbm_test/2.genotype_caller/logs/c.txt 'python  /ancestry-pipeline/ancestry/readcount_caller.py --readcount-dir /scratch1/fs1/dinglab/estorrs/ancestry/executions/gbm_test/1.slate/results --genomes-vcf /storage1/fs1/dinglab/Active/Projects/Ancestry_Data/hg38/all.coding.sorted.02maf.snps.chr.vcf --output /scratch1/

run the below script

In [82]:
fp = os.path.join(execution_dir, 'run_readcount_caller.sh')
write_command_file(bsub_commands, fp)
fp

'/scratch1/fs1/dinglab/estorrs/ancestry/executions/gbm_test/2.genotype_caller/run_readcount_caller.sh'

## step 3: predict ancestry

In [ ]:
execution_dir = os.path.join(run_dir, '3.predict_ancestry')
log_dir = os.path.join(execution_dir, 'logs')
result_dir = os.path.join(execution_dir, 'results')
Path(log_dir).mkdir(parents=True, exist_ok=True)
Path(result_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
def get_ancestry_prediction_command(script, output_dir, sample_vcf_fp):
    return f'python  {script} \
--output-dir {output_dir} {thousand_genomes_vcf} {thousand_genomes_panel} {sample_vcf_fp}'

In [ ]:
volumes = ['/storage1/fs1/dinglab', '/scratch1/fs1/dinglab', '/home/estorrs'] # compute1 needs to know which directories to map, replace /home/<username> if you are not me
exports = ['/miniconda/envs/ancestry/bin'] # need to override compute1 PATH defaults
args = DEFAULT_ARGS.copy()
args

In [ ]:
args['username'] = 'estorrs' # change username if you arent me :) 
args['docker'] = 'estorrs/ancestry-pipeline:0.0.1'
args['queue'] = 'general'
args['mem'] = 100

command = get_ancestry_prediction_command(
    '/ancestry-pipeline/ancestry/ancestry_cli.py',
    result_dir,
    os.path.join(run_dir, '2.genotype_caller', 'results', 'output.vcf'))
   
bsub_commands = batch_bsub_commands([command], 'predict ancestry', log_dir, args, volumes=volumes)
bsub_commands

run the below script

In [ ]:
fp = os.path.join(execution_dir, 'predict_ancestry.sh')
write_command_file(bsub_commands, fp)
fp

## inspect results